# Metadata

**L1 Taxonomy** - Security

**L2 Taxonomy** - Network Security

**Subtopic** - Python Tools for Network Security Analysis

**Use Case** - Develop a Python module that uses the Scapy library to analyze network packets. The module should be able to read packets from a pcap file, filter them based on IP and port, and provide basic statistics such as the number of packets and the most common source and destination IPs.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
scapy==2.6.1
```


# Prompt

## **Problem Overview**

You are tasked with developing a robust and extensible Python module named `PacketAnalyzer`, designed to process and analyze packet capture (pcap) files using the **Scapy** library. The module should offer functionalities to read packets from a given pcap file, filter them based on a specified source/destination IP address and port number, and generate a comprehensive summary of the filtered results.

The module must be developed as a standalone, importable class that can be reused in other security analysis tools or systems. The design should emphasize clarity, extensibility, and separation of concerns. Advanced understanding of packet structures, TCP/IP layers, and Scapy operations will be critical.


## **Input Format**

* A valid path to a `.pcap` file containing recorded network traffic.
* Optional filters provided as keyword arguments:

  * `src_ip`: filter by source IP address (string)
  * `dst_ip`: filter by destination IP address (string)
  * `src_port`: filter by source port number (integer)
  * `dst_port`: filter by destination port number (integer)

These filters may be passed individually or in combination. If no filters are provided, all packets in the pcap file must be considered.


## **Output Format**

A Python dictionary with the following keys:

* `total_packets`: Total number of packets matching the filter criteria.
* `protocol_counts`: A dictionary mapping each protocol (e.g., TCP, UDP, ICMP) to its count.
* `top_source_ips`: A list of tuples representing the top 5 most common source IP addresses along with their occurrence counts, sorted in descending order.
* `top_destination_ips`: A list of tuples representing the top 5 most common destination IP addresses along with their occurrence counts, sorted in descending order.

Example return structure:

```python
{
    "total_packets": 187,
    "protocol_counts": {
        "TCP": 132,
        "UDP": 47,
        "ICMP": 8
    },
    "top_source_ips": [
        ("192.168.1.100", 34),
        ("10.0.0.2", 29),
        ...
    ],
    "top_destination_ips": [
        ("8.8.8.8", 45),
        ("192.168.1.1", 39),
        ...
    ]
}
```


## **Examples**

### Example 1

```python
from packet_analyzer import PacketAnalyzer

analyzer = PacketAnalyzer("sample_traffic.pcap")
stats = analyzer.analyze(src_ip="192.168.1.10", dst_port=80)

print(stats["total_packets"])         # e.g., 56
print(stats["protocol_counts"])       # e.g., {"TCP": 50, "UDP": 6}
print(stats["top_source_ips"])        # e.g., [("192.168.1.10", 56)]
print(stats["top_destination_ips"])   # e.g., [("93.184.216.34", 20), ("172.217.3.110", 18)]
```

### Example 2 (No filters)

```python
analyzer = PacketAnalyzer("traffic.pcap")
stats = analyzer.analyze()

# Should analyze all packets in the file
```


# Requirements

## **Explicit Requirements**

* The class must be named `PacketAnalyzer` and must be defined in a module named `packet_analyzer.py`.
* The constructor must accept the path to the pcap file as a string.
* The method `analyze()` must return a dictionary structured exactly as described in the Output Format.
* You must use Scapy for reading and parsing packets.
* You must handle common Scapy exceptions (e.g., malformed packets) gracefully.
* Source and destination IPs must be extracted only from IP-layer packets (IPv4).
* Ports must be considered only for TCP and UDP layers.
* Protocol detection must differentiate between TCP, UDP, ICMP, and Others.


## **Implicit Requirements**

* The module should not read the pcap file until `analyze()` is called (i.e., lazy loading).
* IPs must be properly validated as IPv4 addresses using `ipaddress` module or equivalent.
* Port filters should be ignored for non-TCP/UDP packets.
* The analyzer should skip packets that don't have the required layers (e.g., Ethernet but no IP).
* All internal logic should be encapsulated in private methods to allow future enhancements.
* The module should be optimized to handle large pcap files efficiently in terms of memory.


## **Function Signature(s) and/or Class Definition(s)**

```python
class PacketAnalyzer:
    def __init__(self, pcap_path: str) -> None:
        """
        Initialize the analyzer with the path to a pcap file.
        """
        pass

    def analyze(
        self,
        src_ip: str = None,
        dst_ip: str = None,
        src_port: int = None,
        dst_port: int = None
    ) -> dict:
        """
        Analyze packets from the pcap file based on given filters.
        Returns a dictionary containing packet statistics.
        """
        pass
```


## **Edge Cases**

* The pcap file does not exist or is unreadable: raise a `FileNotFoundError`.
* The pcap file is empty or contains no valid packets: return all counts as zero.
* No packets match the provided filters: return empty IP lists and protocol counts as empty dict.
* Packets with missing IP/TCP/UDP layers should be skipped and not cause a crash.
* Source or destination IP is invalid (e.g., malformed): raise `ValueError`.


## **Constraints**

* Only the standard library and Scapy (`scapy.all`) may be used.
* Must be compatible with Python 3.7 and above.
* Must not modify the input pcap file in any way.
* The solution must avoid global variables and should be encapsulated in a clean, reusable module.
* Must follow PEP8 and PEP257 guidelines.
* Lines should not exceed 79 characters in length.
* The module should include docstrings for all public classes and methods.

In [ ]:
# code

"""This module implements a PacketAnalyzer class using scapy"""

import ipaddress
from collections import Counter
import scapy.all as scapy


class PacketAnalyzer:
    """
    A class for analyzing network packets from a pcap file using Scapy.
    Supports filtering by source/destination IP and port, and provides
    statistics such as protocol counts and top IP addresses.
    """

    def __init__(self, pcap_path: str) -> None:
        """
        Initialize the analyzer with the path to a pcap file.

        Parameters:
            pcap_path (str): Path to the pcap file.
        """
        self.pcap_path = pcap_path

    def analyze(
        self,
        src_ip: str = None,
        dst_ip: str = None,
        src_port: int = None,
        dst_port: int = None
    ) -> dict:
        """
        Analyze packets from the pcap file based on given filters.

        Parameters:
            src_ip (str, optional): Source IP address to filter.
            dst_ip (str, optional): Destination IP address to filter.
            src_port (int, optional): Source port number to filter.
            dst_port (int, optional): Destination port number to filter.

        Returns:
            dict: A dictionary containing packet statistics.
        """
        try:
            packets = scapy.rdpcap(self.pcap_path)
        except FileNotFoundError as e:
            raise FileNotFoundError(
                f"The file {self.pcap_path} was not found."
            ) from e
        except Exception as e:
            raise OSError(
                f"Error reading the pcap file: {e}"
            ) from e

        # Validate IP addresses, if provided
        if src_ip:
            self._validate_ip(src_ip, "source")
        if dst_ip:
            self._validate_ip(dst_ip, "destination")

        protocol_counts = Counter()
        src_ip_counter = Counter()
        dst_ip_counter = Counter()
        total_packets = 0

        for packet in packets:
            if not self._packet_matches(
                packet, src_ip, dst_ip, src_port, dst_port
            ):
                continue

            if not packet.haslayer(scapy.IP):
                continue

            total_packets += 1
            ip_layer = packet[scapy.IP]

            if packet.haslayer(scapy.TCP):
                protocol_counts["TCP"] += 1
            elif packet.haslayer(scapy.UDP):
                protocol_counts["UDP"] += 1
            elif packet.haslayer(scapy.ICMP):
                protocol_counts["ICMP"] += 1
            else:
                protocol_counts["Others"] += 1

            src_ip_counter[ip_layer.src] += 1
            dst_ip_counter[ip_layer.dst] += 1

        result = {
            "total_packets": total_packets,
            "protocol_counts": dict(protocol_counts),
            "top_source_ips": src_ip_counter.most_common(5),
            "top_destination_ips": dst_ip_counter.most_common(5)
        }

        return result

    def _packet_matches(
        self,
        packet,
        src_ip: str,
        dst_ip: str,
        src_port: int,
        dst_port: int
    ) -> bool:
        """
        Check if a packet matches the specified IP and port filters.

        Parameters:
            packet: Scapy packet object.
            src_ip (str): Source IP address filter.
            dst_ip (str): Destination IP address filter.
            src_port (int): Source port number filter.
            dst_port (int): Destination port number filter.

        Returns:
            bool: True if the packet matches all filters, else False.
        """
        if src_ip:
            if not packet.haslayer(scapy.IP):
                return False
            if packet[scapy.IP].src != src_ip:
                return False

        if dst_ip:
            if not packet.haslayer(scapy.IP):
                return False
            if packet[scapy.IP].dst != dst_ip:
                return False

        if src_port:
            if packet.haslayer(scapy.TCP):
                if packet[scapy.TCP].sport != src_port:
                    return False
            elif packet.haslayer(scapy.UDP):
                if packet[scapy.UDP].sport != src_port:
                    return False
            else:
                return False

        if dst_port:
            if packet.haslayer(scapy.TCP):
                if packet[scapy.TCP].dport != dst_port:
                    return False
            elif packet.haslayer(scapy.UDP):
                if packet[scapy.UDP].dport != dst_port:
                    return False
            else:
                return False

        return True

    def _validate_ip(self, ip_str: str, label: str) -> None:
        """
        Validate an IPv4 address string.

        Parameters:
            ip_str (str): The IP address to validate.
            label (str): Label to indicate whether it's source or destination.

        Raises:
            ValueError: If the IP address is invalid.
        """
        try:
            ipaddress.ip_address(ip_str)
        except ValueError as e:
            raise ValueError(f"Invalid {label} IP address: {ip_str}") from e


In [ ]:
# tests

"""Unit tests for PacketAnalyzer class."""

import unittest
from unittest.mock import patch, MagicMock
from main import PacketAnalyzer


class TestPacketAnalyzer(unittest.TestCase):
    """Test cases for PacketAnalyzer class."""

    def setUp(self):
        """Set up test fixtures."""
        self.valid_pcap_path = "test_traffic.pcap"
        self.analyzer = PacketAnalyzer(self.valid_pcap_path)

    def test_init_valid_path(self):
        """Test initialization with valid pcap path."""
        analyzer = PacketAnalyzer("valid_path.pcap")
        self.assertEqual(analyzer.pcap_path, "valid_path.pcap")

    def test_init_empty_path(self):
        """Test initialization with empty path."""
        analyzer = PacketAnalyzer("")
        self.assertEqual(analyzer.pcap_path, "")

    @patch('main.scapy.rdpcap')
    def test_analyze_file_not_found(self, mock_rdpcap):
        """Test analyze method with non-existent pcap file."""
        mock_rdpcap.side_effect = FileNotFoundError("File not found")

        with self.assertRaises(FileNotFoundError) as context:
            self.analyzer.analyze()

        self.assertIn("was not found", str(context.exception))

    @patch('main.scapy.rdpcap')
    def test_analyze_file_read_error(self, mock_rdpcap):
        """Test analyze method with file read error."""
        mock_rdpcap.side_effect = Exception("Read error")

        with self.assertRaises(OSError) as context:
            self.analyzer.analyze()

        self.assertIn("Error reading the pcap file", str(context.exception))

    @patch('main.scapy.rdpcap')
    def test_analyze_empty_pcap_file(self, mock_rdpcap):
        """Test analyze with empty pcap file."""
        mock_rdpcap.return_value = []

        result = self.analyzer.analyze()

        expected = {
            "total_packets": 0,
            "protocol_counts": {},
            "top_source_ips": [],
            "top_destination_ips": []
        }
        self.assertEqual(result, expected)

    @patch('main.scapy.rdpcap')
    def test_analyze_no_ip_layer_packets(self, mock_rdpcap):
        """Test analyze with packets having no IP layer."""
        mock_packet = MagicMock()
        mock_packet.haslayer.return_value = False
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze()

        expected = {
            "total_packets": 0,
            "protocol_counts": {},
            "top_source_ips": [],
            "top_destination_ips": []
        }
        self.assertEqual(result, expected)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_analyze_tcp_packets(self, mock_tcp, mock_ip, mock_rdpcap):
        """Test analyze with TCP packets."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.return_value.src = "192.168.1.1"
        mock_packet.__getitem__.return_value.dst = "192.168.1.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze()

        self.assertEqual(result["total_packets"], 1)
        self.assertEqual(result["protocol_counts"]["TCP"], 1)
        self.assertEqual(result["top_source_ips"][0], ("192.168.1.1", 1))
        self.assertEqual(result["top_destination_ips"][0], ("192.168.1.2", 1))

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.UDP')
    def test_analyze_udp_packets(self, mock_udp, mock_ip, mock_rdpcap):
        """Test analyze with UDP packets."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_udp
        ]
        mock_packet.__getitem__.return_value.src = "10.0.0.1"
        mock_packet.__getitem__.return_value.dst = "10.0.0.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze()

        self.assertEqual(result["total_packets"], 1)
        self.assertEqual(result["protocol_counts"]["UDP"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.ICMP')
    def test_analyze_icmp_packets(self, mock_icmp, mock_ip, mock_rdpcap):
        """Test analyze with ICMP packets."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_icmp
        ]
        mock_packet.__getitem__.return_value.src = "8.8.8.8"
        mock_packet.__getitem__.return_value.dst = "192.168.1.1"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze()

        self.assertEqual(result["total_packets"], 1)
        self.assertEqual(result["protocol_counts"]["ICMP"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    def test_analyze_other_protocol_packets(self, mock_ip, mock_rdpcap):
        """Test analyze with other protocol packets."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer == mock_ip
        mock_packet.__getitem__.return_value.src = "172.16.0.1"
        mock_packet.__getitem__.return_value.dst = "172.16.0.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze()

        self.assertEqual(result["total_packets"], 1)
        self.assertEqual(result["protocol_counts"]["Others"], 1)

    def test_validate_ip_valid_ipv4(self):
        """Test IP validation with valid IPv4 addresses."""
        # Should not raise any exception
        self.analyzer._validate_ip("192.168.1.1", "source")
        self.analyzer._validate_ip("0.0.0.0", "destination")
        self.analyzer._validate_ip("255.255.255.255", "test")

    def test_validate_ip_invalid_ipv4(self):
        """Test IP validation with invalid IPv4 addresses."""
        with self.assertRaises(ValueError) as context:
            self.analyzer._validate_ip("256.1.1.1", "source")
        self.assertIn("Invalid source IP address", str(context.exception))

        with self.assertRaises(ValueError):
            self.analyzer._validate_ip("192.168.1", "destination")

        with self.assertRaises(ValueError):
            self.analyzer._validate_ip("not.an.ip", "test")

    @patch('main.scapy.rdpcap')
    def test_analyze_with_invalid_src_ip(self, mock_rdpcap):
        """Test analyze with invalid source IP filter."""
        mock_rdpcap.return_value = []

        with self.assertRaises(ValueError) as context:
            self.analyzer.analyze(src_ip="invalid.ip")
        self.assertIn("Invalid source IP address", str(context.exception))

    @patch('main.scapy.rdpcap')
    def test_analyze_with_invalid_dst_ip(self, mock_rdpcap):
        """Test analyze with invalid destination IP filter."""
        mock_rdpcap.return_value = []

        with self.assertRaises(ValueError) as context:
            self.analyzer.analyze(dst_ip="999.999.999.999")
        self.assertIn("Invalid destination IP address",
                      str(context.exception))

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_packet_matches_src_ip_filter(self, mock_tcp, mock_ip,
                                          mock_rdpcap):
        """Test packet filtering by source IP."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.return_value.src = "192.168.1.1"
        mock_packet.__getitem__.return_value.dst = "192.168.1.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(src_ip="192.168.1.1")

        self.assertEqual(result["total_packets"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_packet_matches_dst_ip_filter(self, mock_tcp, mock_ip,
                                          mock_rdpcap):
        """Test packet filtering by destination IP."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.return_value.src = "192.168.1.1"
        mock_packet.__getitem__.return_value.dst = "192.168.1.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(dst_ip="192.168.1.2")

        self.assertEqual(result["total_packets"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_packet_matches_src_port_tcp_filter(self, mock_tcp, mock_ip,
                                                mock_rdpcap):
        """Test packet filtering by TCP source port."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.side_effect = lambda layer: (
            MagicMock(src="192.168.1.1", dst="192.168.1.2")
            if layer == mock_ip
            else MagicMock(sport=80, dport=443)
        )
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(src_port=80)

        self.assertEqual(result["total_packets"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.UDP')
    def test_packet_matches_dst_port_udp_filter(self, mock_udp, mock_ip,
                                                mock_rdpcap):
        """Test packet filtering by UDP destination port."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_udp
        ]
        mock_packet.__getitem__.side_effect = lambda layer: (
            MagicMock(src="10.0.0.1", dst="10.0.0.2")
            if layer == mock_ip
            else MagicMock(sport=53, dport=53)
        )
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(dst_port=53)

        self.assertEqual(result["total_packets"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    def test_packet_no_match_port_filter_no_tcp_udp(self, mock_ip,
                                                    mock_rdpcap):
        """Test port filter with non-TCP/UDP packets."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer == mock_ip
        mock_packet.__getitem__.return_value.src = "192.168.1.1"
        mock_packet.__getitem__.return_value.dst = "192.168.1.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(src_port=80)

        self.assertEqual(result["total_packets"], 0)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_analyze_multiple_filters_combined(self, mock_tcp, mock_ip,
                                               mock_rdpcap):
        """Test analyze with multiple filters combined."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.side_effect = lambda layer: (
            MagicMock(src="192.168.1.100", dst="8.8.8.8")
            if layer == mock_ip
            else MagicMock(sport=12345, dport=80)
        )
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(
            src_ip="192.168.1.100",
            dst_ip="8.8.8.8",
            src_port=12345,
            dst_port=80
        )

        self.assertEqual(result["total_packets"], 1)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_analyze_no_packets_match_filters(self, mock_tcp, mock_ip,
                                              mock_rdpcap):
        """Test analyze when no packets match the filters."""
        mock_packet = MagicMock()
        mock_packet.haslayer.side_effect = lambda layer: layer in [
            mock_ip, mock_tcp
        ]
        mock_packet.__getitem__.return_value.src = "192.168.1.1"
        mock_packet.__getitem__.return_value.dst = "192.168.1.2"
        mock_rdpcap.return_value = [mock_packet]

        result = self.analyzer.analyze(src_ip="10.0.0.1")

        expected = {
            "total_packets": 0,
            "protocol_counts": {},
            "top_source_ips": [],
            "top_destination_ips": []
        }
        self.assertEqual(result, expected)

    @patch('main.scapy.rdpcap')
    @patch('main.scapy.IP')
    @patch('main.scapy.TCP')
    def test_analyze_top_ips_ordering(self, mock_tcp, mock_ip, mock_rdpcap):
        """Test top IPs are correctly ordered by frequency."""
        packets = []
        ips = [
            ("192.168.1.1", "8.8.8.8", 3),  # src, dst, count
            ("192.168.1.2", "8.8.4.4", 5),
            ("192.168.1.3", "1.1.1.1", 1),
        ]

        for src_ip, dst_ip, count in ips:
            for _ in range(count):
                mock_packet = MagicMock()
                mock_packet.haslayer.side_effect = lambda layer: layer in [
                    mock_ip, mock_tcp
                ]
                mock_packet.__getitem__.return_value.src = src_ip
                mock_packet.__getitem__.return_value.dst = dst_ip
                packets.append(mock_packet)

        mock_rdpcap.return_value = packets

        result = self.analyzer.analyze()

        self.assertEqual(result["total_packets"], 9)
        self.assertEqual(result["top_source_ips"][0], ("192.168.1.2", 5))
        self.assertEqual(result["top_source_ips"][1], ("192.168.1.1", 3))
        self.assertEqual(result["top_destination_ips"][0], ("8.8.4.4", 5))


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/c60b8e77-030c-442a-90cc-e6b482678c60?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F84320%2Fview

#### Model code:

```python
import scapy.all as scapy
import ipaddress
from collections import Counter, defaultdict

class PacketAnalyzer:
    def __init__(self, pcap_path: str) -> None:
        """
        Initialize the analyzer with the path to a pcap file.
        
        Parameters:
        pcap_path (str): Path to the pcap file.
        """
        self.pcap_path = pcap_path

    def analyze(self, src_ip: str = None, dst_ip: str = None, src_port: int = None, dst_port: int = None) -> dict:
        """
        Analyze packets from the pcap file based on given filters.
        
        Parameters:
        src_ip (str, optional): Source IP address to filter.
        dst_ip (str, optional): Destination IP address to filter.
        src_port (int, optional): Source port number to filter.
        dst_port (int, optional): Destination port number to filter.
        
        Returns:
        dict: A dictionary containing packet statistics.
        """
        # Lazy load the packets when analyze is called
        try:
            packets = scapy.rdpcap(self.pcap_path)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"The file {self.pcap_path} was not found.") from e
        except Exception as e:
            raise OSError(f"Error reading the pcap file: {e}") from e

        # Validate IP addresses, if provided
        if src_ip:
            try:
                ipaddress.ip_address(src_ip)
            except ValueError:
                raise ValueError(f"Invalid source IP address: {src_ip}")

        if dst_ip:
            try:
                ipaddress.ip_address(dst_ip)
            except ValueError:
                raise ValueError(f"Invalid destination IP address: {dst_ip}")

        # Initialize counters
        protocol_counts = Counter()
        src_ip_counter = Counter()
        dst_ip_counter = Counter()
        total_packets = 0

        # Filter and process packets
        for packet in packets:
            # Basic packet filtering based on provided criteria
            if not self._packet_matches(packet, src_ip, dst_ip, src_port, dst_port):
                continue

            total_packets += 1
            # Increment protocol counts
            if packet.haslayer(scapy.IP):
                if packet.haslayer(scapy.TCP):
                    protocol_counts['TCP'] += 1
                elif packet.haslayer(scapy.UDP):
                    protocol_counts['UDP'] += 1
                elif packet.haslayer(scapy.ICMP):
                    protocol_counts['ICMP'] += 1
                else:
                    protocol_counts['Others'] += 1

                # Increment source and destination IP counts
                src_ip_counter[packet[scapy.IP].src] += 1
                dst_ip_counter[packet[scapy.IP].dst] += 1
                # <Issue>: There’s no surrounding if packet.haslayer(scapy.IP) guard here. If a malformed or non-IP packet slips through packet_matches(), this would raise an exception, violating the requirement.

        # Prepare the result
        result = {
            "total_packets": total_packets,
            "protocol_counts": dict(protocol_counts), # <Issue>: The problem's example output only shows protocols that were present. This code may include keys like "Others": 0 if no such packets existed. This contradicts the expected structure "exactly as described".
            "top_source_ips": src_ip_counter.most_common(5),
            "top_destination_ips": dst_ip_counter.most_common(5)
        }
        return result

    def _packet_matches(self, packet, src_ip, dst_ip, src_port, dst_port):
        """
        Check if the packet matches the provided filters.
        """
        # Filtering based on IP and Port
        # <Issue>: Although some haslayer() checks are present, packet[scapy.IP].src and packet[scapy.IP].dst are accessed directly without confirming the presence of the IP layer inside the main loop, violating the skip requirement.
        if src_ip and (not packet.haslayer(scapy.IP) or packet[scapy.IP].src != src_ip):
            return False
        if dst_ip and (not packet.haslayer(scapy.IP) or packet[scapy.IP].dst != dst_ip):
            return False
        
        if src_port and (not packet.haslayer(scapy.TCP) or packet.haslayer(scapy.UDP)) and packet.sport != src_port:
            return False
        if dst_port and (not packet.haslayer(scapy.TCP) or packet.haslayer(scapy.UDP)) and packet.dport != dst_port:
            return False
        # <Issue>: This logic attempts to apply sport/dport filtering without first ensuring the packet has either a TCP or UDP layer. This violates the requirement that port filters should be ignored for non-TCP/UDP packets.
        return True
```